In [ ]:
import os

import numpy
import pandas as pd
import pickle
import seaborn as sns
from textwrap import wrap
from matplotlib import pyplot as plt
from scipy import signal
from scipy.stats import mode

from hera_cal.redcal import get_reds

from simpleredcal.plot_utils import clipped_heatmap
from simpleredcal.red_likelihood import fltBad, group_data, makeCArray, \
red_ant_sep
from simpleredcal.red_utils import find_flag_file, find_nearest, find_rel_df, \
find_zen_file, get_bad_ants, JD2LSTPATH, match_lst, RESPATH

In [ ]:
%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = (12, 8)

import matplotlib as mpl

plot_figs = False
if plot_figs:
    mpl.rcParams['figure.dpi'] = 300

mpl.rc('font',**{'family':'serif','serif':['cm']})
mpl.rc('text', usetex=True)
mpl.rc('text.latex', preamble=r'\usepackage{amssymb} \usepackage{amsmath}')

max_title_length = 100

In [ ]:
JD = 2458098.43869
pol = 'ee'
statistic = 'median' # {'mean', 'median'} used for averaging visibilities
rel_dir_path = os.path.join(RESPATH, 'rel_dfs')

clip_pctile = 98 # for heatmaps to set vmax

In [ ]:
import matplotlib as mpl

plot_figs = False
if plot_figs:
    mpl.rcParams['figure.dpi'] = 300

mpl.rc('font',**{'family':'serif','serif':['cm']})
mpl.rc('text', usetex=True)
mpl.rc('text.latex', preamble=r'\usepackage{amssymb} \usepackage{amsmath}')

save_fig_dir = '/Users/matyasmolnar/Desktop/Thesis/CHAP-4/FIGS/'

# Difference in adjacent visibilities as an estimate of noise

In [ ]:
stat = getattr(numpy, statistic)

In [ ]:
zen_fn = find_zen_file(JD)
bad_ants = get_bad_ants(zen_fn)
flags_fn = find_flag_file(JD, 'first')

hdraw, RedG, cMData = group_data(zen_fn, pol, None, None, bad_ants, flags_fn)
cData_1 = cMData.filled()

###  Selecting visibilities for a given baseline type and frequency

In [ ]:
freq_channel = 620

In [ ]:
reds = fltBad(get_reds(hdraw.antpos, pols=[pol]), bad_ants)
bl_types = RedG[:, 0]
slct_bl_type_id = mode(bl_types)[0][0] # selecting modal value for baseline type
slct_bl_type = reds[slct_bl_type_id][0]
slct_idxs = numpy.where(bl_types == slct_bl_type_id)[0]
print('Selecting visibilities with baseline type {} that are redundant with '\
      'baseline {}, of which there are {}'.format(slct_bl_type_id, slct_bl_type, \
      slct_idxs.size))

In [ ]:
slct_vis_t = numpy.squeeze(cData_1[numpy.ix_([freq_channel], \
                                             numpy.arange(cData_1.shape[1]), slct_idxs)])

In [ ]:
vis_amp_t = numpy.abs(slct_vis_t)
hlsts = hdraw.lsts*12/numpy.pi
last0 = round(hlsts[0], 3)

fig, ax = plt.subplots(figsize=(8, 7))

ax.plot(vis_amp_t, alpha=0.5, linewidth=1)
ax.plot(numpy.median(vis_amp_t, axis=1), linewidth=2, label='median', color='purple')
ax.plot(numpy.mean(vis_amp_t, axis=1), linewidth=2, label='mean', color='orange')

ax.set_xlabel('Time integration')
ax.set_ylabel('Visibility amplitude')
ax.set_title('\n'.join(wrap('Amplitudes for visibilities with baselines redundant to {} at '\
    'frequency channel {} on JD {} at LAST {} h'.format(slct_bl_type, freq_channel, int(JD), \
                                                      last0), 80)))
ax.legend(loc=1)

fig.tight_layout()
plt.show()

In [ ]:
df_t = pd.DataFrame(vis_amp_t).stack().reset_index()
df_t.rename(columns={'level_0': 'time_int', 'level_1': 'bl', 0: 'vis_amp'}, inplace=True)

fig, ax = plt.subplots(figsize=(11, 7))

ax = sns.lineplot(x='time_int', y='vis_amp', data=df_t, ci='sd', linewidth=3)

ax.set_xlabel('Time integration')
ax.set_ylabel('Visibility amplitude')
ax.set_title('\n'.join(wrap(r'Median amplitude for visibilities with baselines redundant to '\
    '{} at frequency channel {} on JD {} at LAST {}, with 1$\sigma$ confidence interval '\
    'shown'.format(slct_bl_type, freq_channel, int(JD), last0), max_title_length)))

fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))

ax.plot(numpy.angle(slct_vis_t), alpha=0.6)

ax.set_xlabel('Time integration')
ax.set_ylabel('Visibility phase')
ax.set_title('\n'.join(wrap('Phases for visibilities with baselines redundant to {} at '\
    'frequency channel {} on JD {} at LAST {}'\
                         .format(slct_bl_type, freq_channel, int(JD), last0), \
                                 80)))

fig.tight_layout()
plt.show()

###  Selecting visibilities for a given baseline type and time integration

In [ ]:
tint = 20
slct_vis_f = numpy.squeeze(cData_1[numpy.ix_(numpy.arange(cData_1.shape[0]), [tint], slct_idxs)])

In [ ]:
vis_amp_f = numpy.abs(slct_vis_f)
last0 = round(hlsts[tint], 3)

ytop = numpy.nanpercentile(vis_amp_f, 98.5)

fig, ax = plt.subplots(figsize=(11, 7))

ax.plot(vis_amp_f, alpha=0.3, linewidth=1)
ax.plot(numpy.median(vis_amp_f, axis=1), linewidth=1.5, label='median', color='purple', alpha=0.8)
ax.plot(numpy.mean(vis_amp_f, axis=1), linewidth=1.5, label='mean', color='orange', alpha=0.8)

ax.set_ylim(-0.01, ytop)
ax.set_xlabel('Frequency channel')
ax.set_ylabel('Visibility amplitude')
ax.set_title('\n'.join(wrap('Amplitudes for visibilities with baselines redundant to {} at '\
    'time integration {} on JD {} at LAST {} h'.format(slct_bl_type, int(tint), int(JD), \
                                                      last0), max_title_length)))

ax.legend(loc=1)

fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))

ax.plot(numpy.angle(slct_vis_f), alpha=0.5, linewidth=1)

ax.set_xlabel('Time integration')
ax.set_ylabel('Visibility phase')
ax.set_title('\n'.join(wrap('Phases for visibilities with baselines redundant to {} at '\
    'time integration {} on JD {} at LAST {} h'\
                         .format(slct_bl_type, tint, int(JD), last0), \
                                 max_title_length)))

fig.tight_layout()
plt.show()

### Distribution of data

#### Visibility amplitude fitting

##### Gaussian vs Cauchy fitting

In [ ]:
from scipy.stats import cauchy, norm

frange = numpy.arange(freq_channel,freq_channel+5)
vis_amp_samples = vis_amp_f[frange]

cmap = plt.get_cmap('tab10')

fig, ax = plt.subplots(figsize=(8, 5))

for i, vis_amp_sample in enumerate(vis_amp_samples):
    ax.hist(vis_amp_sample, bins=numpy.arange(0.005, 0.04, 0.002), density=True, alpha=0.2, color=cmap(i))
    
    xmin, xmax = plt.xlim()
    x = numpy.linspace(xmin, xmax, 1000)
    mu, std = norm.fit(vis_amp_sample)
    med, gamma = cauchy.fit(vis_amp_sample)
    g = norm.pdf(x, mu, std)
    c = cauchy.pdf(x, med, gamma)
    
    ax.plot(x, g, linewidth=2, label=f'f{frange[i]} '+r'$\mathcal{N}$'+'; $\mu$ = {:.4f}, $\sigma$ = {:.4f}'.format(mu, std), \
             color=cmap(i))
    ax.plot(x, c, linewidth=2, label=f'f{frange[i]} '+r'$\mspace{3mu}\mathcal{C}\mspace{4.2mu}$'+'; $x$ = {:.4f}, $\gamma$ = {:.4f}'.format(med, gamma), \
             linestyle='dashed', color=cmap(i))

ax.set_xlabel(r'$|V|$')
ax.set_ylabel('Density')
ax.legend(loc='upper right')

ax.set_title('\n'.join(wrap('Amplitudes for visibilities with baselines redundant to {} at '\
    'frequency channels {}-{} and time integration {} on JD {} at LAST {} h'.\
    format(slct_bl_type, frange[0], frange[-1], int(tint), int(JD), last0), 80)))

fig.tight_layout()
plt.show()

##### Cumulative distribution

In [ ]:
frange = numpy.arange(freq_channel,freq_channel+5)
vis_amp_samples = vis_amp_f[frange]
f_idx = 2

cmap = plt.get_cmap('tab10')

fig, ax = plt.subplots(figsize=(11, 7))

for i, vis_amp_sample in enumerate([vis_amp_samples[f_idx]]):
    ax.hist(vis_amp_sample, bins=numpy.arange(0.005, 0.04, 0.002), density=True, alpha=0.2, \
             color=cmap(i), cumulative=True)
    
    xmin, xmax = plt.xlim()
    x = numpy.linspace(xmin, xmax, 1000)
    mu, std = norm.fit(vis_amp_sample)
    med, gamma = cauchy.fit(vis_amp_sample)
    g = norm.cdf(x, mu, std)
    c = cauchy.cdf(x, med, gamma)
    
    ax.plot(x, g, linewidth=2, label='f{} Gaussian; $\mu$ = {:.4f}, $\sigma$ = {:.4f}'.format(frange[i], mu, std), \
            color=cmap(i))
    ax.plot(x, c, linewidth=2, label='f{} Cauchy; $x$ = {:.4f}, $\gamma$ = {:.4f}'.format(frange[i], med, gamma), \
            linestyle='dashed', color=cmap(i))

ax.legend(loc='best')
ax.set_title('\n'.join(wrap('Amplitudes for visibilities with baselines redundant to {} at '\
    'frequency channels {}-{} and time integration {} on JD {} at LAST {}'.format(slct_bl_type, frange[0], \
    frange[-1], int(tint), int(JD), last0), 80)))
ax.set_xlabel('Visibility amplitude')

fig.tight_layout()
plt.show()

##### Showing extreme case where Gaussian suffers

In [ ]:
frange = numpy.arange(700,702)
vis_amp_samples = vis_amp_f[frange]

cmap = plt.get_cmap('tab10')

fig, ax = plt.subplots(figsize=(8, 5))

for i, vis_amp_sample in enumerate(vis_amp_samples):
    ax.hist(vis_amp_sample, bins=numpy.arange(0.005, 0.3, 0.002), density=True, alpha=0.2, color=cmap(i))
    
    xmin, xmax = plt.xlim()
    x = numpy.linspace(xmin, xmax, 10000)
    mu, std = norm.fit(vis_amp_sample)
    med, gamma = cauchy.fit(vis_amp_sample)
    g = norm.pdf(x, mu, std)
    c = cauchy.pdf(x, med, gamma)
    
    ax.plot(x, g, linewidth=2, label=f'f{frange[i]} '+r'$\mathcal{N}$'+'; $\mu$ = {:.4f}, $\sigma$ = {:.4f}'.format(mu, std), color=cmap(i))
    ax.plot(x, c, linewidth=2, label=f'f{frange[i]} '+r'$\mspace{3mu}\mathcal{C}\mspace{4.2mu}$'+'; $x$ = {:.4f}, $\gamma$ = {:.4f}'.format(med, gamma), linestyle='dashed', color=cmap(i))
    
ax.legend(loc='best')
ax.set_title('\n'.join(wrap('Amplitudes for visibilities with baselines redundant to {} at '\
    'frequency channels {}-{} and time integration {} on JD {} at LAST {}'.format(slct_bl_type, frange[0], \
    frange[-1], int(tint), int(JD), last0), 80)))
ax.set_xlabel(r'$|V|$')
ax.set_ylabel('Density')
ax.set_xlim(0, 0.3)

fig.tight_layout()
plt.show()

In [ ]:
# showing extreme case where both Gaussian and Cauchy have poor fits, 
# but the Cauchy performs worse than the Gaussian suffers

frange = numpy.arange(383,386) # all of these are bad channels
vis_amp_samples = vis_amp_f[frange]

cmap = plt.get_cmap('tab10')

fig, ax = plt.subplots(figsize=(11, 7))

for i, vis_amp_sample in enumerate(vis_amp_samples):
    ax.hist(vis_amp_sample, bins=numpy.arange(numpy.floor(vis_amp_sample.min()), numpy.ceil(vis_amp_sample.max()), 0.5), \
             density=True, alpha=0.2, color=cmap(i))
    
    xmin, xmax = plt.xlim()
    x = numpy.linspace(0, 100, 10000)
    mu, std = norm.fit(vis_amp_sample)
    med, gamma = cauchy.fit(vis_amp_sample)
    g = norm.pdf(x, mu, std)
    c = cauchy.pdf(x, med, gamma)
    
    ax.plot(x, g, linewidth=2, label='f{} Gaussian; $\mu$ = {:.4f}, $\sigma$ = {:.4f}'.format(frange[i], mu, std), \
                                                                                               color=cmap(i))
    ax.plot(x, c, linewidth=2, label='f{} Cauchy; $x$ = {:.4f}, $\gamma$ = {:.4f}'.format(frange[i], med, gamma), \
                                                                                           linestyle='dashed', color=cmap(i))
    
ax.legend()
ax.set_title('\n'.join(wrap('Amplitudes for visibilities with baselines redundant to {} at '\
    'frequency channels {}-{} and time integration {} on JD {} at LAST {}'.format(slct_bl_type, frange[0], \
                                                                   frange[-1], int(tint), int(JD), last0), 80)))
ax.set_xlabel('Visibility amplitude')

fig.tight_layout()
plt.show()

In [ ]:
# zooming in on channel 383

frange = 383
vis_amp_sample = vis_amp_f[frange]
print(vis_amp_sample)

cmap = plt.get_cmap('tab10')

fig, ax = plt.subplots(figsize=(11, 7))

ax.hist(vis_amp_sample, bins=numpy.arange(0.0, 5, 0.2), density=True, alpha=0.5)

xmin, xmax = plt.xlim()
x = numpy.linspace(0, 10, 10000)
mu, std = norm.fit(vis_amp_sample)
med, gamma = cauchy.fit(vis_amp_sample)
g = norm.pdf(x, mu, std)
c = cauchy.pdf(x, med, gamma)

ax.plot(x, g, linewidth=2, label='f{} Gaussian; $\mu$ = {:.4f}, $\sigma$ = {:.4f}'.format(frange, mu, std))
ax.plot(x, c, linewidth=2, label='f{} Cauchy; $x$ = {:.4f}, $\gamma$ = {:.4f}'.format(frange, med, gamma), linestyle='dashed')

ax.set_xlim(0, 5)
ax.set_xlabel('Visibility amplitude')
ax.set_title('\n'.join(wrap('Amplitudes for visibilities with baselines redundant to {} at '\
    'frequency channel {} and time integration {} on JD {} at LAST {}'.format(slct_bl_type, frange, \
                                                                       int(tint), int(JD), last0), 80)))
ax.legend(loc='best')

fig.tight_layout()
plt.show()

#### Complex visibility 2D fitting

In [ ]:
frange = numpy.arange(freq_channel,freq_channel+5)[3] # only selecting one frequency for time being
slct_vis_s = slct_vis_f[frange]

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))

ax.scatter(slct_vis_s.real, slct_vis_s.imag)

ax.set_xlabel('Real')
ax.set_ylabel('Imaginary')
ax.set_title('\n'.join(wrap('Visibilities for baselines redundant to {} at '\
    'frequency channel {} and time integration {} on JD {} at LAST {}'.format(slct_bl_type, frange, \
    int(tint), int(JD), last0), 80)))

fig.tight_layout()
plt.show()

In [ ]:
import functools
from scipy.optimize import minimize

In [ ]:
def gauss2d(x, y, x0, y0, sigma):
    """2D Gaussian distribution with same variance across x and y"""
    x_inner = numpy.square(x - x0) / (numpy.square(sigma))
    y_inner = numpy.square(y - y0) / (numpy.square(sigma))
    return 1/(2*numpy.pi*sigma**2) * numpy.exp(-0.5*(x_inner + y_inner))

In [ ]:
def gauss2dll(x, y, params):
    """2D Gaussian negative log-likelihood MLE with same variance across x and y"""
    x0, y0, sigma = params
    nll = x.size*numpy.log(2*numpy.pi*sigma**4)/2 + (numpy.square(x - x0) + \
          numpy.square(y - y0)).sum()/(2*sigma**2)
    return nll

In [ ]:
ff = functools.partial(gauss2dll, slct_vis_s.real, slct_vis_s.imag)
res = minimize(ff, (0, 0, 0.1))
print(res['x'])

In [ ]:
side = numpy.linspace(-0.05, 0.05, 1001)
x, y = numpy.meshgrid(side, side)
z = gauss2d(x, y, *res['x'])

fig, ax = plt.subplots(figsize=(9, 9))

ax.pcolormesh(x, y, z)
ax.scatter(slct_vis_s.real, slct_vis_s.imag, color='orange')

# Mean and standard deviation added as a red dot and circle
ax.plot(res['x'][0], res['x'][1], 'o', color='red')
circ = plt.Circle((res['x'][0], res['x'][1]), res['x'][2], color='red', fill=False)
ax.add_artist(circ)

ax.set_xlabel('Real')
ax.set_ylabel('Imaginary')
ax.set_title('\n'.join(wrap('2D Gaussian PDF to complex visibilities for baselines redundant to {} at '\
    'frequency channel {} and time integration {} on JD {} at LAST {}'.format(slct_bl_type, frange, \
                                                                       int(tint), int(JD), last0), 80)))

fig.tight_layout()
plt.show()

#### Goodness of fit

In [ ]:
vis_amp_sample = vis_amp_f[freq_channel]
from scipy.stats import kstest

##### KS-test

In [ ]:
kstest(vis_amp_sample, 'norm')

In [ ]:
kstest(vis_amp_sample, 'cauchy')

##### Log-likelihood

Gaussian negative log-likelihood:

$$ - \ln{(\mathcal{L}^G)} = \frac{N}{2} \ln \left( 2\pi\sigma^2 \right) + \frac{1}{2\sigma^2} \sum_{i=1}^N \left( x_i - \mu \right)^2 $$

Cauchy negative log-likelihood:

$$ - \ln{(\mathcal{L}^C)} = N \ln{\left( \pi \gamma \right)} + \sum_{i=1}^N \ln{ \left( 1 + \left( \frac{x_i - x}{\gamma} \right)^2 \right)} $$

In [ ]:
def logl_g(obs, mean, std):
    nll = obs.size*numpy.log(2*numpy.pi*std**2)/2 + numpy.square(obs - mean).sum()/(2*std**2)
    return -nll

def logl_c(obs, med, hwhm):
    nll = obs.size*numpy.log(numpy.pi*hwhm) + (numpy.log(numpy.square((obs - med)/hwhm) + 1)).sum()
    return -nll

In [ ]:
logl_g_res = numpy.empty(vis_amp_f.shape[0])
logl_c_res = numpy.empty_like(logl_g_res)
l_ratio = numpy.empty_like(logl_g_res)

for i, vis_amp_s in enumerate(vis_amp_f):
    if not numpy.isnan(vis_amp_s).all():
        mu, sigma = norm.fit(vis_amp_s)
        med, gamma = cauchy.fit(vis_amp_s)
        logl_g_i = logl_g(vis_amp_s, mu, sigma)
        logl_c_i = logl_c(vis_amp_s, med, gamma)
        
    else:
        logl_g_i = numpy.nan
        logl_c_i = numpy.nan
    
    logl_g_res[i] = logl_g_i
    logl_c_res[i] = logl_c_i
    l_ratio[i] = -2 * (logl_g_i - logl_c_i)

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))

ax.plot(-logl_g_res, label='Gaussian', alpha=0.6)
ax.plot(-logl_c_res, label='Cauchy', alpha=0.6)
# ax.plot(l_ratio, label='Likelihood ratio')

ax.set_xlabel('Frequency channel')
ax.set_ylabel('Negative log-likelihood')
ax.set_title('\n'.join(wrap('Comparison of negative log-likelihoods for Gaussian and Cauchy fittings across frequencies for '\
    'visibilities with baselines redundant to {} at time integration {} on JD {} at LAST {}'.\
    format(slct_bl_type, int(tint), int(JD), last0), 100)))
ax.legend()

fig.tight_layout()
plt.show()

##### Student's t-distribution

Non-standardized probability density function:
$$	f(t; \nu, x_0, \gamma) = \frac{\Gamma \left( \frac{\nu + 1}{2} \right)}{\sqrt{\pi \nu} \Gamma \left( \frac{\nu}{2} \right) \gamma } \left(1 + \frac{1}{\nu} \left( \frac{t - x_0}{\gamma} \right)^2 \right) ^{-\frac{\nu + 1}{2}} $$

Log-likelihood:
$$ -\log{\left( \mathcal{L}^T \right)} \left( \nu, x_0, \gamma \right)  = - N \ln{\left( \Gamma \left( \frac{\nu + 1}{2} \right) \right)}  + \frac{N}{2} \ln{ \left( \pi \nu \right) } + N \ln{ \left( \Gamma \left( \frac{\nu}{2} \right) \right) } + N \ln{ \left( \gamma \right) } + \frac{\nu+1}{2} \sum_i^N \ln{ \left( 1 + \frac{1}{\nu} \left( \frac{t_i - x_0}{\gamma} \right)^2 \right) } $$

In [ ]:
from scipy.stats import t as student_t

loc = 0
gamma = 2
nus = [1, 2, 3, 4, 5, 20]
x = numpy.linspace(-20, 20, 10000)

fig, ax = plt.subplots(figsize=(7, 5))

for nu in nus:
    dist = student_t.pdf(x, nu, loc, gamma)
    ax.plot(x, dist, label=r'$\nu$ = {}, $x_0=0$, $\gamma = 2$'.format(nu))
    
ax.plot(x, norm.pdf(x, 0, 2), ls='--', label=r'$\mathcal{N}$, $\mu=0$, $\sigma = 2$')

ax.set_xlim(-10, 10)
ax.set_xlabel('$t$')
ax.set_ylabel(r'$P(t \mid \nu, x, \gamma )$')
ax.set_title('Student\'s $t$ Distribution')
ax.legend(loc='best')

fig.tight_layout()
# plt.savefig(os.path.join(save_fig_dir, 't_dists.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
from scipy.special import gamma as gammaf

def logl_t(obs, nu, loc, scale):
    ll = obs.size*(numpy.log(gammaf(0.5*(nu + 1))) - 0.5*numpy.log(numpy.pi * nu) - \
                   numpy.log(gammaf(0.5*nu)) - numpy.log(scale)) - \
                   0.5*(nu + 1)*(numpy.log(1 + (1/nu)*numpy.square((obs - loc)/scale))).sum() 
    return ll

In [ ]:
logl_g_res = numpy.empty(vis_amp_f.shape[0])
logl_t_res = {nu:numpy.empty_like(logl_g_res) for nu in nus}

for i, vis_amp_s in enumerate(vis_amp_f):
    if not numpy.isnan(vis_amp_s).all():
        mu, sigma = norm.fit(vis_amp_s)
        logl_g_res[i] = logl_g(vis_amp_s, mu, sigma)
        
        for nu in nus:
            df, med, gamma = student_t.fit(vis_amp_s, f0=nu)
            logl_t_res[nu][i] = logl_t(vis_amp_s, nu, med, gamma)
    else:
        logl_g_res[i] = numpy.nan
        for nu in nus:
            logl_t_res[nu][i] = numpy.nan

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))

ax.plot(-logl_g_res, label='Gaussian', alpha=0.5, linewidth=1)
ax.plot(-logl_c_res, label='Cauchy', alpha=0.5, linewidth=1)

for nu in [nus[2]]:
    ax.plot(-logl_t_res[nu], label='Student\'s '+r'$t$, $\nu={}$'.format(nu), alpha=0.5, linewidth=1)
    
ax.set_xlabel('Frequency Channel')
# ax.set_ylabel('Negative log-likelihood')
ax.set_ylabel(r'$-\ln(\mathcal{L})$')
ax.set_xlim(0, 1023)
ax.legend(loc='best')

ax.set_title('\n'.join(wrap('Comparison of negative log-likelihoods for Gaussian and Cauchy fittings across frequencies for '\
                         'visibilities with baselines redundant to {} '\
                         'at time integration {} on JD {} at LAST {}'.format(slct_bl_type, int(tint), int(JD), last0), 100)))

fig.tight_layout()
plt.show()

## Adjacent time integrations

For a given baseline, one can compare the visibilities between one time integration and the next to get an estimate of the system noise, since the difference between adjacent visibilities will encapsulate both the noise and the difference in the observed sky (due to a slight drift). The statistics of the difference in visibilities therefore provides an upper bound on the noise of the visibilities.

We take the standard deviation of the different between visibilities adjacent in time for an entire dataset, as a proxy for the noise. This must first be done per baseline and per frequency. The noise for redundant baselines can then be combined through propagation of error considerations.

n.b. the variance of a complex random variable $z$ is equal to the sum of the variances of its real and imaginary parts:

$$ \mathrm{Var}[z] = \mathrm{Var}[\Re(z)] + \mathrm{Var}[\Im(z)] $$

which we use when calculating the standard deviation of complex visibilities.

In [ ]:
vis_diffs = numpy.empty((cData_1.shape[0], cData_1.shape[1]-1, \
                         cData_1.shape[2]), dtype=complex)
noise_std = numpy.empty((cData_1.shape[0], cData_1.shape[2]))
stat_vis_amp = numpy.empty_like(noise_std)
for bl in range(cData_1.shape[2]):
    for freq in range(cData_1.shape[0]):
        vdiff = numpy.asarray([t - s for s, t in zip(cData_1[freq, :, bl], \
                                                     cData_1[freq, 1:, bl])])
        vis_diffs[freq, :, bl] = vdiff
        noise_std[freq, bl] = numpy.sqrt(numpy.var(vdiff.real) + \
                                             numpy.var(vdiff.imag))
        stat_vis_amp[freq, bl] = stat(numpy.abs(cData_1[freq, :, bl]))

We add errors in quadrature when considering the noise across a redundant baseline type, assuming an error covariance of zero (independent measurements and uncorrelated errors).

$$\sigma_{\text{red_group}} = \frac{1}{N} \sqrt{\sum_i{\sigma_i^2}} $$

In [ ]:
no_unq_bls = numpy.unique(bl_types).size
red_noise = numpy.empty((cData_1.shape[0], no_unq_bls))
red_vis_amp = numpy.empty_like(red_noise)

for bl_type in range(no_unq_bls):
    group_idxs = numpy.where(bl_types == bl_type)[0]
    grouped_noise = noise_std[:, group_idxs]
    grouped_vis_amp = stat_vis_amp[:, group_idxs]
    red_noise[:, bl_type] = numpy.sqrt(numpy.sum(numpy.square(grouped_noise), axis=1)) \
                            / grouped_noise.shape[1]
    red_vis_amp[:, bl_type] = stat(grouped_vis_amp, axis=1)

### Single baseline: {{reds[slct_bl_type][0]}}

#### Single frequency: channel {{freq_channel}}

In [ ]:
bl_grp_id = 0 # ID of baseline within its group
bl_id = slct_idxs[bl_grp_id]

vis_bl_t = cData_1[:, :, bl_id]
print('From the {} baselines of type {} selected, only consider baseline {}\n'\
      .format(slct_idxs.size, slct_bl_type, reds[slct_bl_type_id][bl_grp_id]))

vis_diffs_t = vis_diffs[:, :, bl_id]
noise_std_t = noise_std[freq_channel, bl_id]
stat_vis_amp_t = stat_vis_amp[freq_channel, bl_id]
print('Upper bound on noise, by comparing adjacent visibilities in time for '\
      'frequency channel {} is {}, which is {}% of the {} visibility amplitude'.\
      format(freq_channel, round(noise_std_t, 5), round(100*noise_std_t/stat_vis_amp_t, 1), \
             statistic))

#### Noise per frequency

In [ ]:
vis_bl_tf = cData_1[..., bl_id]
vis_diffs_tf = vis_diffs[..., bl_id]
noise_std_tf = noise_std[..., bl_id]
stat_vis_amp_tf = stat_vis_amp[..., bl_id]

In [ ]:
fig, ax = clipped_heatmap(numpy.abs(vis_diffs_tf).transpose(), 'Time integration', xoffset=0)

ax.set_title('\n'.join(wrap('Residual between adjacent visibilities (in time) for baselines '\
    'redundant to {} on JD {} at LAST {}'.format(slct_bl_type, int(JD), last0), max_title_length)))

fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))

ax.plot(noise_std_tf)

ax.set_yscale('log')
ax.set_xlabel('Frequency channel')
ax.set_ylabel('Log-noise')
ax.set_title('\n'.join(wrap('Log of the standard deviation of residuals between adjacent '\
    'visibilities (in time) for baselines redundant to {} on JD {} at LAST {}'\
    .format(slct_bl_type, int(JD), last0), max_title_length)))

fig.tight_layout()
plt.show()

In [ ]:
frac_noise_tf = noise_std_tf/stat_vis_amp_tf

fig, ax = plt.subplots(figsize=(11, 7))

ax.plot(frac_noise_tf)

ax.set_xlabel('Frequency channel')
ax.set_ylabel('Fractional noise')
ax.set_title('\n'.join(wrap('Standard deviation of residuals between adjacent visibilities '\
    '(in time) for baselines redundant to {}, divided by the {} visibility amplitude on JD '\
    '{} at LAST {}'.format(slct_bl_type, int(JD), last0, statistic), max_title_length)))
ytop = numpy.ceil(numpy.nanpercentile(frac_noise_tf, clip_pctile))
ax.set_ylim(bottom=0, top=ytop)

fig.tight_layout()
plt.show()

### Redundant baseline group

In [ ]:
print('Considering baselines of type {}'.format(slct_bl_type))

In [ ]:
vis_diffs_tfb = cData_1[..., slct_idxs]
noise_std_tfb = noise_std[..., slct_idxs]
stat_vis_amp_tfb = stat_vis_amp[..., slct_idxs]

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))

for bl in range(noise_std_tfb.shape[1]):
    ax.plot(noise_std_tfb[:, bl], alpha=0.3, linewidth=1)
    
ax.set_yscale('log')
ax.set_xlabel('Frequency channel')
ax.set_ylabel('Log-noise')
ax.set_title('\n'.join(wrap('Log of the standard deviation of residuals between adjacent '\
    'visibilities (in time) for baselines of type {} on JD {} at LAST {}'\
    .format(slct_bl_type, int(JD), last0), max_title_length)))
ax.set_ylim(1e-4, 1e0)

fig.tight_layout()
plt.show()

In [ ]:
frac_noise_tfb = noise_std_tfb/stat_vis_amp_tfb

fig, ax = plt.subplots(figsize=(11, 7))

for bl in range(noise_std_tfb.shape[1]):
    ax.plot(frac_noise_tfb[:, bl], alpha=0.3, linewidth=1)
    
ax.set_xlabel('Frequency channel')
ax.set_ylabel('Fractional noise')
ax.set_title('\n'.join(wrap('Standard deviation of residuals between adjacent visibilities '\
    '(in time) divided by the {} visibility amplitude, for baselines of type {} on JD {} '\
    'at LAST {}'.format(statistic, slct_bl_type, int(JD), last0), max_title_length)))
ytop = numpy.ceil(numpy.nanpercentile(frac_noise_tfb, clip_pctile))
ax.set_ylim(bottom=0, top=ytop)

fig.tight_layout()
plt.show()

In [ ]:
fig, ax = clipped_heatmap((noise_std_tfb/stat_vis_amp_tfb).transpose(), 'Baseline', xoffset=0)

ax.set_title('\n'.join(wrap('Standard deviation of residuals between adjacent visibilities '\
    '(in time) divided by the {} visibility amplitude, for baselines in redundant group {} '\
    'on JD {} at LAST {}'.format(statistic, slct_bl_type, int(JD), last0), max_title_length)))

fig.tight_layout()
plt.show()

#### Noise across redundant group

In [ ]:
red_noise_g = red_noise[:, slct_bl_type_id]

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))

ax.plot(red_noise_g)

ax.set_yscale('log')
ax.set_xlabel('Frequency channel')
ax.set_ylabel('Log-noise')
ax.set_title('\n'.join(wrap('Log of the combined standard deviation of residuals between '\
    'adjacent visibilities (in time) for baseline group of type {} on JD {} at LAST {}'\
    .format(slct_bl_type, int(JD), last0), max_title_length)))

fig.tight_layout()
plt.show()

In [ ]:
frac_noise_redg = red_noise_g/stat(stat_vis_amp[:, slct_idxs], axis=1)

fig, ax = plt.subplots(figsize=(11, 7))

ax.plot(frac_noise_redg)

ax.set_xlabel('Frequency channel')
ax.set_ylabel('Fractional noise')
ax.set_title('\n'.join(wrap('Standard deviation of residuals between adjacent visibilities '\
    '(in time) divided by the {} visibility amplitude, for baselines redundant with {} on '\
    'JD {} at LAST {}'.format(statistic, slct_bl_type, int(JD), last0), max_title_length)))
ytop = numpy.ceil(numpy.nanpercentile(frac_noise_tfb, clip_pctile))
ax.set_ylim(bottom=0, top=1)

fig.tight_layout()
plt.show()

In [ ]:
# plotting individual and combined noise together

fig, ax = plt.subplots(figsize=(11, 7))

for bl in range(noise_std_tfb.shape[1]):
    ax.plot(frac_noise_tfb[:, bl], alpha=0.3, linewidth=1)
ax.plot(frac_noise_redg, linewidth=1.5, color='purple', label='Combined noise', alpha=0.8)

ax.set_xlabel('Frequency channel')
ax.set_ylabel('Fractional noise')
ax.set_title('\n'.join(wrap('Standard deviation of residuals between adjacent visibilities '\
    '(in time) divided by the {} visibility amplitude, for baselines redundant with {} on '\
    'JD {} at LAST {}'.format(statistic, slct_bl_type, int(JD), last0), max_title_length)))
ax.set_ylim(bottom=0, top=3)
ax.legend(loc=1)

fig.tight_layout()
plt.show()

#### Noise power spectrum

In [ ]:
chan_bottom = 400
chan_top = 700

Examining the power spectrum of the noise, between frequency channels {{chan_bottom}} and {{chan_top}}

In [ ]:
resolution = hdraw.freqs[1] - hdraw.freqs[0] # Hz
f, Pxx_spec = signal.periodogram(frac_noise_redg[numpy.arange(chan_bottom,chan_top)], \
                                 resolution, window='blackmanharris', detrend=False, \
                                 scaling='spectrum')

In [ ]:
fig, ax =plt.subplots(figsize=(7, 5))

ax.semilogy(f, numpy.sqrt(Pxx_spec))

ax.set_xlabel('Delay [s]')
ax.set_ylabel('Power spectrum [V RMS]')

fig.tight_layout()
plt.show()

### Noise across baselines

In [ ]:
no_bls_g = numpy.array([numpy.array(group).shape[0] for group in reds])
red_bl_lengths = numpy.sqrt(numpy.sum(numpy.square(red_ant_sep(RedG, hdraw.antpos)), axis=1))

In [ ]:
dict_bl_info = {'Baseline type':[i[0] for i in reds], \
                'Number of baselines': no_bls_g, \
                'Baseline length': red_bl_lengths,
                'Redundant baselines': reds}
df_bl_info = pd.DataFrame.from_dict(dict_bl_info)

In [ ]:
df_bl_info.head()

In [ ]:
frac_noise = red_noise/red_vis_amp

fig, ax = clipped_heatmap(frac_noise.transpose(), 'Baseline type', xoffset=0)

ax.set_title('\n'.join(wrap('Standard deviation of residuals between adjacent visibilities '\
    '(in time) divided by the {} visibility amplitude, for all redundant baseline types, '\
    'on JD {} at LAST {}'.format(statistic, int(JD), last0), max_title_length)))

fig.tight_layout()
plt.show()

The lower baseline type IDs correspond to shorter baselines, with baseline length increase (and number of baselines decreasing) with as the baseline type ID increases. We find that the shorter baselines higher frequency channels have the lowest noise out of our dataset.

Note that the closure phase analysis found that the optimal frequency ranges for are $110-136$ MHz and $151-173$ MHz, corresponding to channels $102-369$ and $522-748$, respectively (see [HERA Memo 54](http://reionization.org/wp-content/uploads/2018/11/hera-memo-54.pdf))

#### Normalizing by number of baselines in each group

In [ ]:
nrm_no_bl_frac_noise = frac_noise*numpy.sqrt(no_bls_g[None, :])

In [ ]:
fig, ax = clipped_heatmap(nrm_no_bl_frac_noise.transpose(), 'Baseline type', xoffset=0)

ax.set_title('\n'.join(wrap('Standard deviation of residuals between adjacent visibilities '\
    '(in time) divided by the {} visibility amplitude, for all redundant baseline types, '\
    'on JD {} at LAST {}, multiplied by the square root of the number of baselines for each '\
    'type'.format(statistic, int(JD), last0), max_title_length)))

fig.tight_layout()
plt.show()

From this heatmap, if the number of of baselines in each redundant group is taken into account (multiply the noise by $\sqrt{N}$), the noise seems to be of around the same magnitude across all groups.

#### Further normalize by baseline length in each group

In [ ]:
nrm_no_bl_length_frac_noise = nrm_no_bl_frac_noise/red_bl_lengths[None, :]

In [ ]:
fig, ax = clipped_heatmap(nrm_no_bl_length_frac_noise.transpose(), 'Baseline type', xoffset=0)

ax.set_title('\n'.join(wrap('Standard deviation of residuals between adjacent visibilities '\
    '(in time) divided by the {} visibility amplitude, for all redundant baseline types, '\
    'on JD {} at LAST {}, multiplied by the square root of the number of baselines and '\
    'divided by the baseline length, for each type'.format(statistic, int(JD), last0), max_title_length)))

fig.tight_layout()
plt.show()

## Same LAST on different JDs

An alternative estimate of the noise is by looking at visibilities from different JDs that match in LAST, although the visibilities on different JDs may vastly differ due to the variability of the instrumental gains and other effects (e.g. ionosphere).

In [ ]:
JD_2a= match_lst(JD, 2458099) # finding the JD_time of the zen_file
# that matches the LAST of the first dataset used
zen_fn2a = find_zen_file(JD_2a)
bad_ants2a = get_bad_ants(zen_fn2a)
flags_fn2a = find_flag_file(JD_2a, 'first')

In [ ]:
last_df = pd.read_pickle(JD2LSTPATH)

next_row = numpy.where(last_df['JD_time'] == JD_2a)[0][0] + 1
JD_2b = last_df.iloc[next_row]['JD_time']
zen_fn2b = find_zen_file(JD_2b)
bad_ants2b = get_bad_ants(zen_fn2b)
flags_fn2b = find_flag_file(JD_2b, 'first')

In [ ]:
last1 = last_df[last_df['JD_time'] == JD]['LASTs'].values[0]
last2 = last_df[last_df['JD_time'] == JD_2a]['LASTs'].values[0]
_, offset = find_nearest(last2, last1[0])

In [ ]:
_, _, cMData = group_data(zen_fn2a, pol, None, None, bad_ants2a, flags_fn2a)
cData_2a = cMData.filled()[:, offset:, :]

_, _, cMData = group_data(zen_fn2b, pol, None, None, bad_ants2b, flags_fn2b)
cData_2b = cMData.filled()[:, :offset, :]

In [ ]:
cData_2 = numpy.concatenate((cData_2a, cData_2b), axis=1)
del cData_2a, cData_2b

###  Selecting visibilities for a given baseline type and frequency

In [ ]:
slct_vis_j = numpy.squeeze(cData_2[numpy.ix_([freq_channel], \
                                             numpy.arange(cData_2.shape[1]), slct_idxs)])
vis_amp_j = numpy.abs(slct_vis_j)
lastj = round(last2[offset], 3)

fig, ax = plt.subplots(figsize=(11, 7))

ax.plot(vis_amp_j, alpha=0.5, linewidth=1)
ax.plot(numpy.median(vis_amp_j, axis=1), linewidth=2, color='purple', label='median')

ax.set_xlabel('Time integration')
ax.set_ylabel('Visibility amplitude')
ax.set_title('\n'.join(wrap('Amplitudes for visibilities with baselines redundant to {} at '\
    'frequency channel {} on JD {} at LAST {}'.format(slct_bl_type, freq_channel, int(JD_2a), \
    lastj), max_title_length)))
ax.legend()

fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))

ax.plot(numpy.median(vis_amp_t, axis=1), label=str(JD))
ax.plot(numpy.median(vis_amp_j, axis=1), label=str(JD_2a))

ax.set_xlabel('Time integration')
ax.set_ylabel('Visibility amplitude')
# ax.set_title('Median visibility amplitude on separate JD days but same LAST')
ax.set_title('\n'.join(wrap('Median visibility amplitudes for baselines redundant to {} at '\
    'frequency channel {} on JDs {} and {} at LAST {}'.format(slct_bl_type, freq_channel, \
    int(JD), int(JD_2a), lastj), max_title_length)))
ax.legend()

fig.tight_layout()
plt.show()

### Noise diagnostics

In [ ]:
vis_diffs_j = numpy.empty((cData_2.shape[0], cData_2.shape[1]-1, \
                           cData_2.shape[2]), dtype=complex)
noise_std_j = numpy.empty((cData_2.shape[0], cData_2.shape[2]))
stat_vis_amp_j = numpy.empty_like(noise_std_j)
for bl in range(cData_2.shape[2]):
    for freq in range(cData_2.shape[0]):
        vdiff = numpy.asarray([t - s for s, t in zip(cData_2[freq, :, bl], \
                                                     cData_2[freq, 1:, bl])])
        vis_diffs_j[freq, :, bl] = vdiff
        noise_std_j[freq, bl] = numpy.sqrt(numpy.var(vdiff.real) + \
                                           numpy.var(vdiff.imag))
        stat_vis_amp_j[freq, bl] = stat(numpy.abs(cData_2[freq, :, bl]))

In [ ]:
red_noise_j = numpy.empty((cData_2.shape[0], no_unq_bls))
red_vis_amp_j = numpy.empty_like(red_noise_j)

for bl_type in range(no_unq_bls):
    group_idxs = numpy.where(bl_types == bl_type)[0]
    grouped_noise = noise_std_j[:, group_idxs]
    grouped_vis_amp = stat_vis_amp_j[:, group_idxs]
    red_noise_j[:, bl_type] = numpy.sqrt(numpy.sum(numpy.square(grouped_noise), axis=1)) \
                            / grouped_noise.shape[1]
    red_vis_amp_j[:, bl_type] = stat(grouped_vis_amp, axis=1)

#### Noise across a redundant group

In [ ]:
red_noise_gj = red_noise_j[:, slct_bl_type_id]

fig, ax = plt.subplots(figsize=(11, 7))

ax.plot(red_noise_gj/stat(stat_vis_amp_j[:, slct_idxs], axis=1), label='LAST', alpha=0.8)
ax.plot(red_noise_g/stat(stat_vis_amp[:, slct_idxs], axis=1), label='time', alpha=0.8)

ax.set_xlabel('Frequency channel')
ax.set_ylabel('Fractional noise')
ax.set_title('\n'.join(wrap('Standard deviation of residuals between adjacent visibilities '\
    '(in time [JD {0}] and LAST [JDs {0} and {1}]) divided by the {2} visibility amplitude, '\
    'for baselines redundant with {3} at LAST {4}'.format(int(JD), int(JD_2a), statistic, \
    slct_bl_type, lastj), max_title_length)))
ax.set_ylim(bottom=0, top=1)
ax.legend()

fig.tight_layout()
plt.show()

The fractional noise estimates using both methods match well.

In [ ]:
frac_noise_j = red_noise_j/red_vis_amp_j

fig, ax = clipped_heatmap(frac_noise_j.transpose(), 'Baseline type', xoffset=0)

ax.set_title('\n'.join(wrap('Standard deviation of residuals between adjacent visibilities '\
    '(in LAST [JDs {} and {}]) divided by the {} visibility amplitude, for all  redundant '\
    'baseline types, at LAST {}'.format(int(JD), int(JD_2a), statistic, lastj), \
    max_title_length)))

fig.tight_layout()
plt.show()

In [ ]:
noise_diff = (frac_noise - frac_noise_j).transpose()

fig, ax = clipped_heatmap(noise_diff, 'Baseline type', cmap='bwr', center=0, xoffset=0)

ax.set_title('\n'.join(wrap('Difference in the noise estimated from visibilities '\
    'adjacent in time [JD {0}] and LAST [JDs {0} and {1}] for all baseline types at LAST {2}'\
    .format(int(JD), int(JD_2a), lastj), max_title_length)))

fig.tight_layout()
plt.show()

Negative (positive) noise difference values in blue (red) indicate that the noise estimates from using visibilities adjacent in time are lower (higher) than that estimated using visibilities at the same LAST on different JDs.

# Noise in visibility solutions

We can look at the scatter of the visibility solutions from our relative redundant calibration of raw visibilities, to see if this calibration step improves on the intrinsic noise found in Section 1 of this notebook.

In [ ]:
distribution = 'gaussian'

In [ ]:
rel_df_path = find_rel_df(JD, pol, distribution, rel_dir=rel_dir_path)
rel_df = pd.read_pickle(rel_df_path)

freq_chans = rel_df.index.get_level_values('freq').unique().values
time_ints = rel_df.index.get_level_values('time_int').unique().values

## Visibility amplitudes for test channel {{freq_channel}}

In [ ]:
rel_vis = numpy.empty((freq_chans.size, time_ints.size, no_unq_bls), dtype=complex)
for f, freq in enumerate(freq_chans):
    rel_vis_comps, _ = numpy.split(rel_df.loc[freq, :].values[:, 5:-2]\
                                   .astype(float), [no_unq_bls*2,], axis=1)
    for tint in time_ints:
        rel_vis[f, tint, :] = makeCArray(rel_vis_comps[tint, :])

In [ ]:
rel_vis_amp_f = numpy.abs(rel_vis[freq_channel, ...])

fig, ax = plt.subplots(figsize=(11, 7))

ax.plot(rel_vis_amp_f, alpha=0.5)

ax.set_xlabel('Time integration')
ax.set_ylabel('Visibility amplitude')
ax.set_title('\n'.join(wrap('Amplitudes for visibility solutions for all redundant '\
    'baseline groups at frequency channel {} on JD {} at LAST {}'\
    .format(freq_channel, int(JD), last0), max_title_length)))

fig.tight_layout()
plt.show()

## Noise across frequencies and baseline groups

We calculate the noise for each frequency and baseline group, by taking the standard deviation of the visibility solutions over all time integrations.

In [ ]:
rel_noise_std = numpy.empty((freq_chans.size, no_unq_bls))
stat_rel_vis_amp = numpy.empty_like(rel_noise_std)

for freq in range(freq_chans.size):
    for bl_g in range(no_unq_bls):
        rel_vis_fb = rel_vis[freq, :, bl_g]
        rel_vis_diffs = numpy.asarray([t - s for s, t in zip(rel_vis_fb, rel_vis_fb[1:])])
        rel_noise_std[freq, bl_g] = numpy.sqrt(numpy.var(rel_vis_diffs.real) + \
                                               numpy.var(rel_vis_diffs.imag))
        stat_rel_vis_amp[freq, bl_g] = stat(numpy.abs(rel_vis_fb))

In [ ]:
rel_frac_noise = rel_noise_std/stat_rel_vis_amp

fig, ax = clipped_heatmap(rel_frac_noise.transpose(), 'Baseline type')

ax.set_title('\n'.join(wrap('Standard deviation of residuals between adjacent visibility '\
    'solutions (in time), divided by the {} visibility solution amplitude, for all '\
    'redundant baseline types, on JD {} at LAST {}'\
    .format(statistic, int(JD), last0), max_title_length)))

fig.tight_layout()
plt.show()

### Normalized by the number of baselines in each group

In [ ]:
nrm_no_bl_rel_frac_noise = rel_frac_noise*numpy.sqrt(no_bls_g[None, :])

fig, ax = clipped_heatmap(nrm_no_bl_rel_frac_noise.transpose(), 'Baseline type')

ax.set_title('\n'.join(wrap('Standard deviation of residuals between adjacent visibility '\
    'solutions (in time), divided by the {} visibility solution amplitude, for all '\
    'redundant baseline types, on JD {} at LAST {}, multiplied by the square root of the '\
    'number of baselines for each type'.format(statistic, int(JD), last0), max_title_length)))

fig.tight_layout()
plt.show()

### Noise comparison with raw visibilities

In [ ]:
rel_noise_diff = (rel_frac_noise/frac_noise[freq_chans]).transpose()

fig, ax = clipped_heatmap(rel_noise_diff, 'Baseline type', cmap='bwr', center=0)

ax.set_title('\n'.join(wrap('Difference between the estimates of the noise for visibility '\
    'solutions and that from raw visibilities adjacent in time and LAST for all baseline '\
    'types on JD {} at LAST {}'.format(int(JD), last0), max_title_length)))

fig.tight_layout()
plt.show()

Red (blue) regions correspond to baseline groups and frequencies where the noise from raw visibilities is lower (higher) than that for visibility solutions calculated from relative redundant calibration.